In [1]:
from backend_logic import *
import pandas as pd

getProject("2637865")

{'_id': '5d6f6cde1a578fae725bdda2',
 'domain': 'lifeproof-store.ru',
 'projectId': 2637865,
 'service': 'SERM',
 'status': 'active',
 'strategy': 'Максимум трафика'}

In [ ]:
projects_df = pd.read_excel('projects.xlsx')
projects = projects_df.to_dict('records')
projects

In [ ]:
client = MongoClient('mongodb://192.168.10.77:27017/')
db = client.Offers
collection = db.apraProject
for project in projects:
#     collection.insert_one(project)